In [1]:
import sys
!{sys.executable} -m pip install opencv-contrib-python --upgrade

In [2]:
def convolution(oldimage, kernel):
    image_h = oldimage.shape[0]
    image_w = oldimage.shape[1]
    
    
    kernel_h = kernel.shape[0]
    kernel_w = kernel.shape[1]
    
    if(len(oldimage.shape) == 3):
        image_pad = np.pad(oldimage, pad_width=(
            (kernel_h // 2, kernel_h // 2),(kernel_w // 2, 
            kernel_w // 2),(0,0)), mode='constant', 
            constant_values=0).astype(np.float32)
    elif(len(oldimage.shape) == 2):
        image_pad = np.pad(oldimage, pad_width=(
            (kernel_h // 2, kernel_h // 2),
            (kernel_w // 2, kernel_w // 2)), mode='constant', constant_values=0).astype(np.float32)
    
    
    h = kernel_h // 2
    w = kernel_w // 2
    
    image_conv = np.zeros(image_pad.shape)
    
    for i in range(h, image_pad.shape[0]-h):
        for j in range(w, image_pad.shape[1]-w):
            #sum = 0
            x = image_pad[i-h:i-h+kernel_h, j-w:j-w+kernel_w]
            x = x.flatten()*kernel.flatten()
            image_conv[i][j] = x.sum()
    h_end = -h
    w_end = -w
    
    if(h == 0):
        return image_conv[h:,w:w_end]
    if(w == 0):
        return image_conv[h:h_end,w:]
    return image_conv[h:h_end,w:w_end]

In [3]:
def GaussianBlurImage(image, mult=1):
    gaussian_filter= np.array([[1, 4, 6, 4, 1],
                             [4,16,24,16,4],
                             [6,24,36,24,6],
                             [4,16,24,16,4],
                             [1, 4, 6, 4, 1]])/256 * mult
    
    
    im_filtered = np.zeros_like(image, dtype=np.float32)
    for c in range(3):
        im_filtered[:, :, c] = convolution(image[:, :, c], gaussian_filter)
    return (im_filtered.astype(np.uint8))

In [4]:
def pyrD(image):
    #Convolve with kernel
    blurred = GaussianBlurImage(image)
    #Remove every even-numbered row and column
    down = blurred[::2,::2]
    return down

In [5]:
def pyrU(image):
    #Upsample
    shp = (2*image.shape[0],2*image.shape[1],image.shape[2])
    out = np.zeros(np.array(shp),dtype=image.dtype)
    out[::2,::2] = image
    #Convolve
    up = GaussianBlurImage(out, 4)
    return up

In [6]:
import cv2 as cv
import numpy as np,sys
A = cv.imread('./data/apple.png')
B = cv.imread('./data/orange.png')
# generate Gaussian pyramid for A
G = A.copy()
gpA = [G]
for i in range(6):
    G = pyrD(G)
    gpA.append(G)
# generate Gaussian pyramid for B
G = B.copy()
gpB = [G]
for i in range(6):
    G = pyrD(G)
    gpB.append(G)
# generate Laplacian Pyramid for A
lpA = [gpA[5]]
for i in range(5,0,-1):
    GE = pyrU(gpA[i])
    if gpA[i-1].shape[0] % 2 == 1:
        GE = GE[:-1,:,:]
    if gpA[i-1].shape[1] % 2 == 1:
        GE = GE[:,:-1,:]    
    L = cv.subtract(gpA[i-1],GE)
    lpA.append(L)
# generate Laplacian Pyramid for B
lpB = [gpB[5]]
for i in range(5,0,-1):
    GE = pyrU(gpB[i])
    if gpB[i-1].shape[0] % 2 == 1:
        GE = GE[:-1,:,:]
    if gpB[i-1].shape[1] % 2 == 1:
        GE = GE[:,:-1,:]
    L = cv.subtract(gpB[i-1],GE)
    lpB.append(L)
# Now add left and right halves of images in each level
LS = []
for la,lb in zip(lpA,lpB):
    rows,cols,dpt = la.shape
    ls = np.hstack((la[:,0:cols//2], lb[:,cols//2:]))
    LS.append(ls)
# now reconstruct
ls_ = LS[0]
for i in range(1,6):
    ls_ = pyrU(ls_)
    if LS[i].shape[0] % 2 == 1:
        ls_ = ls_[:-1,:,:]
    if LS[i].shape[1] % 2 == 1:
        ls_ = ls_[:,:-1,:]
    ls_ = cv.add(ls_, LS[i])

# image with direct connecting each half
real = np.hstack((A[:,:cols//2],B[:,cols//2:]))

cv.imshow("Original",np.hstack([A,B]))
cv.imshow("Result",np.hstack([ls_,real]))
cv.waitKey(0)
cv.destroyAllWindows()